In [43]:
import numpy as np
import scipy.signal as signal
import os
import scipy.io as scio
import pandas as pd

import MST_Graph
import VeFC_mst

import VeFC_ex2fd as cal_VeFC
import VnFC_ex2fd as cal_VFC
import Idiff

### 输入输出和参数设置

输入数据

In [26]:
# 输入数据路径
GROUP1_DATA2021_FOLDER = 'data2021/Class1-Parent'#亲子组
GROUP2_DATA2021_FOLDER = 'data2021/Class2-Stranger'#陌生人组
GROUP1_DATA2023_FOLDER = 'data2023/Class1-Parent'#亲子组
GROUP2_DATA2023_FOLDER = 'data2023/Class2-Stranger'#陌生人组
GROUP1_DATAALL_FOLDER = 'data_all/Class1-Parent'#亲子组
GROUP2_DATAALL_FOLDER = 'data_all/Class2-Stranger'#陌生人组

# 输入数据结构：
# 每个数据文件包含6种拼图场景，其中前面的1/2表示儿童拼或者成人拼，
# 后面的1/2/3表示单独/陪伴/合作,只关注3（合作）
sit_c1 = 'ca11' #儿童拼-单独
sit_a1 = 'ca21' #成人拼-单独
sit_c2 = 'ca12' #儿童拼-陪伴
sit_a2 = 'ca22' #成人拼-陪伴
sit_c3 = 'ca13' #儿童拼-合作
sit_a3 = 'ca23' #成人拼-合作
# 每种情况包含4个cell文件，分别对应4个频段：1-3，3-6，6-9，9-12
delta,theta,alpha,belta=0,1,2,3
freq_all=['delta','theta','alpha','belta']
# 每个cell文件的格式为通道数*时间点数*被试数
# 通道数: 其中1-28为儿童，29-56为成人，通过subc选择类型
# 28个通道及其对应的索引为：
channame_full = ['Fp1','Fp2','Fz','F3','F4','F7','F8','FC1','FC2','FC5','FC6','Cz','C3','C4','T7','T8','CP1','CP2','CP5','CP6','Pz','P3','P4','P7','P8','Oz','O1','O2']
id_chan_all = list(range(0,28))
# 26个通道及其对应的索引为：
channame_test = ['Fp1','Fp2','Fz','F3','F4','F7','F8','FC1','FC2','FC5','FC6','Cz','C3','C4','CP1','CP2','CP5','CP6','Pz','P3','P4','P7','P8','Oz','O1','O2']
id_chan_test = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27]

输出文件

In [37]:
# 输出数据所要导出到的Excel文件
output_excel='output/Diff_dyad_mst_all.xlsx'
# df=pd.DataFrame() #构造原始数据文件
# df.to_excel(output_excel)

参数设置

In [64]:
# 参数设置
chan_num = 28 #通道数
edge_num = int(chan_num*(chan_num-1)/2) #边数
Nuptran_indices = cal_VeFC.uptran_mask(chan_num) #矩阵上三角列向量的位置
Euptran_indices = cal_VeFC.uptran_mask(edge_num) #矩阵上三角列向量的位置
Vtype='uptran'

# itype指边连接强度的计算方式，'p','c','d'分别代表：PLV复指数、corr相关、delta做差
itype_all=['e','c','d']
itype=itype_all[0]
# corr_type指边与边之间共波动特性的计算方式，'cos','pearson','delta'分别代表：
# 余弦相似度，皮尔逊相关系数，直接做差
corr_type_all=['cos','pearson','delta']
corr_type=corr_type_all[0]

tp=2000

sit=sit_a3 #拼图场景
freq=theta #频段
subc='adult'  #对象类型 

### 计算nFC方法的结果，用以比较

In [65]:
# 函数cal_Iden对结果进行验证
# 比较eFC矩阵对于某种模态mode的识别能力
# 将所有模态对应的index拼接在一起
dyad_all1 = Idiff.file_path_infd(GROUP1_DATAALL_FOLDER)

# eFC矩阵向量化的结果记为VeFC
i=0
VnFC_mode=[]
trn_mode=[] #每种模态trail的数目

for dyad in dyad_all1:
    # 导入mat数据
    data_mat = scio.loadmat(dyad)
    # data_mat的格式为情况*1*频段
    # 选择目标模态对应的数据，ts的格式为被试数*时间点数*通道数
    ts = data_mat[sit][0][freq].T
    # 时间点选择
    ts = ts[:,0:tp,:]
    # 群体subc选择
    if subc=='child': # 如果subject的类型为 child
        ts_subc = ts[:,:,0:28]
    elif subc=='adult': # 如果subject的类型为 adult
        ts_subc = ts[:,:,28:56]
    # 计算eFC的向量形式
    if Vtype=='all': #eFC列向量VeFC
        VnFC_file = cal_VFC.VnFC_all(ts_subc,itype)
    elif Vtype=='uptran': #eFC上三角列向量VeFC_uptran
        VnFC_file = cal_VFC.VnFC_upt(ts_subc,Nuptran_indices,itype)
    # 拼接同一个mode对应的所有文件中的eFC上三角列向量
    # 结果的维度为列向量长度*总的被试数
    trn_mdi=VnFC_file.shape[1]
    if i==0:
        VnFC_mode=VnFC_file
        trn_mode=trn_mdi
        i=1
    else:
        VnFC_mode=np.c_[VnFC_mode,VnFC_file]
        trn_mode=np.c_[trn_mode,trn_mdi]

I_diff,I_self,I_other,I_self_mean,per_change,per_diff=Idiff.cal_Iden(VnFC_mode,trn_mode,'pearsonr')
print('%.2f'%I_diff,'%.2f'%I_self_mean,'%.2f'%I_other)
# 保留两位小数并记录数据
I_concat = [I_diff,I_self_mean,I_other,per_change,per_diff]
I_nFC = np.around(I_concat,2)

8.63 0.72 0.63


### 方法：使用最小生成树MST对VeFC序列矩阵降维

参考：

https://cloud.tencent.com/developer/article/2353167

https://cloud.tencent.com/developer/article/2354144

https://blog.csdn.net/chandelierds/article/details/91357784

#### 1、计算出所有eFC矩阵的均值矩阵eFC_mean_all

In [66]:
dyad_all = Idiff.file_path_infd(GROUP1_DATAALL_FOLDER)
eFC_mean_all = VeFC_mst.eFC_all_mean(dyad_all,sit,freq,subc,corr_type,tp)

In [67]:
# 取绝对值
eFC_mean_abs = np.abs(eFC_mean_all)
# 取1-原值：原值越大则1-原值越小，用于之后的最小生成树MST
eFC_abs_inv = 1-eFC_mean_abs

#### 2、根据均值矩阵eFC_mean_all计算最小生成树MST对应的矩阵编号edge_mst_idx

In [68]:
edge_list1,edge_list2=[],[]
for edge1 in range(1,edge_num):
    for edge2 in range(edge1):
        # 边列表1，2
        edge_list1 = np.append(edge_list1,edge1)
        edge_list2 = np.append(edge_list2,edge2)

In [69]:
eFC_Graph = MST_Graph.Graph()
for i in range(edge_list1.shape[0]):
    e1,e2=edge_list1[i],edge_list2[i]
    e1,e2=int(e1),int(e2)
    eFC_Graph.add_edge(e1,e2,-eFC_mean_abs[e1,e2])

In [70]:
edge_mst_list = eFC_Graph.prim()
edge_mst_ary = np.array(edge_mst_list)[:]
edge_mst_idx = edge_mst_ary[:,:2]

#### 3、计算MST降维后的eFC向量汇总矩阵VeFC_mst_alld

In [71]:
VeFC_mst_alld = VeFC_mst.VeFC_mst_all(dyad_all1,sit,freq,subc,corr_type,tp,
                             edge_mst_idx)

In [72]:
I_diff,I_self,I_other,I_self_mean,per_change,per_diff=Idiff.cal_Iden(VeFC_mst_alld,trn_mode,'pearsonr')
print('%.2f'%I_diff,'%.2f'%I_self_mean,'%.2f'%I_other)
# 保留两位小数并记录数据
I_concat = [I_diff,I_self_mean,I_other,per_change,per_diff]
I_eFC_mst = np.around(I_concat,2)

36.69 0.69 0.32


### 导出结果至excel文件

In [73]:
# 创建一个新的工作表，命名为：场景+对象类型+频段
new_sheet_name = sit+'-'+subc+'-'+freq_all[freq] 
# # 比较各个分支的结果
# new_sheet_name = itype+'-'+corr_type
excel_writer = pd.ExcelWriter(output_excel, mode='a',engine='openpyxl')

In [74]:
# 拼接所有条件下的I_concat数据
I_all = np.c_[I_eFC_mst,I_nFC]
I_all = I_all.T

# 转换为DataFrame格式
df_Iall = pd.DataFrame(I_all)
# 添加列名、索引
df_Iall.columns=['I_diff','I_self','I_other','Change_rate','Diff_rate'] #设置列名
df_Iall.index=['eFC','nFC']

# 保存至excel
df_Iall.to_excel(excel_writer, sheet_name=new_sheet_name)
excel_writer.save()

C:\Users\吴诚成\AppData\Local\Temp\ipykernel_19260\147880195.py:13: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()
